In [1]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from utils import create_nested_directories
import json
import stat

import os
from bs4 import Tag
import mdformat
import lxml

README_FILENAME = "README.md"


In [47]:
class ProjectCreator:

    directory: str = ""
    credentials = None
    response = None

    def __init__(self) -> None:
        self.load_credentials()

    def load_credentials(self):
        """The file must contain a JSON object with the following structure:
        {
            "headers": {
                "User-Agent": "your_user_agent"
            },
            "cookies": {
                "cookie_name": "cookie_value"
            }
        }
        """
        with open("account.json") as f:
            data = json.load(f)

        required_fields = ["headers", "cookies"]

        if not data:
            raise ValueError("No data found in the credentials file")

        for field in required_fields:
            if field not in data:
                raise ValueError(f"Missing field {field} in the credentials file")

        if not data["headers"]["User-Agent"]:
            raise ValueError("Missing User-Agent in the headers field")

        if not data["cookies"]:
            raise ValueError("Missing cookies in the cookies field")

        self.credentials = data

    def save_to_file(self, content: str | bytes, filename: str, executable=False):
        with open(filename, "w") as f:
            f.write(content)

        if executable:
            file = os.stat(filename)

            os.chmod(filename, file.st_mode | stat.S_IXUSR)

    def download_image(self, url: str, filename: str, force_download=False):
        """ Download an image from a given URL and save it to the images directory """
        file = os.path.join("images", filename)

        if os.path.exists(file) and not force_download:
            return

        res = requests.get(url)

        res.raise_for_status()

        with open(file, "wb") as f:
            f.write(res.content)

    def make_request(self, url: str, raise_exception=True):
        res = requests.get(
            url, headers=self.credentials["headers"], cookies=self.credentials["cookies"], allow_redirects=True
        )

        if raise_exception:
            res.raise_for_status()

        self.response = res

        return res

    def create_directories(self, directory: str):
        os.makedirs(directory, exist_ok=True)

    def find_directory_and_create_dirs(self, soup: BeautifulSoup):
        container = soup.find("div", id="task-num-0")

        lis = container.find_all("li")

        for li in lis:
            if "Directory:" in li.text:
                directory = li

        if not directory:
            raise ValueError("No directory found in the project")

        self.directory = directory.text.split(":")[1].strip()

        return True

    def create_readme(self, soup: BeautifulSoup, force_create=False):
        container_desc = soup.find("div", {"id": "project-description"})

        self.find_directory_and_create_dirs(soup)

        filename = os.path.join(self.directory, README_FILENAME)

        if os.path.exists(filename) and not force_create:
            return

        image_tags: list[Tag] = soup.find_all("img")
        link_tags: list[Tag] = container_desc.find_all("a")

        if len(image_tags) > 0:
            if not os.path.exists("images"):
                os.mkdir("images")

            for tag in image_tags:
                src = tag.get("src")

                filename = src.split("/")[-1].split("?")[0]

                self.download_image(src, filename)

                tag["src"] = filename

                tag.attrs["src"] = filename

        if len(link_tags) > 0:
            for tag in link_tags:
                href = tag.get("href")

                if not href.startswith("http"):
                    href = "https://intranet.hbtn.io" + href

                link = self.scrap_link(href)

                tag["href"] = link

        self.save_to_file(md(container_desc.prettify()), filename)

        mdformat.file(filename)

    def create_task_files(self, tasks: list[Tag]):
        for task in tasks:
            task_title = task.find("h3").text
            task_content = task.find("div", attrs={"class": "panel-body"}).prettify()

            specifications = task.find("div", {"class": "list-group-item"})

            if not specifications:
                continue

            requirements = specifications.find_all("li")

            filename = ""
            directory = ""

            for req in requirements:
                if "Directory:" in req.text:
                    directory = req.text.split(":")[1].strip()

                if "File:" in req.text:
                    filename = req.text.split(":")[1].strip()

            if not directory:
                directory = task_title.replace(" ", "_").lower()

            if not filename:
                filename = task_title.replace(" ", "_") + ".py"

            if not os.path.exists(directory):
                create_nested_directories(directory)

            task_content = mdformat.text(md(task_content))

            self.save_to_file(f'{chr(35)}!/usr/bin/env python3\n"""{task_content}"""', os.path.join(directory, filename), executable=True)

    def scrap_link(self, link: str):
        res = self.make_request(link, raise_exception=False)

        return res.url

    def scrap_the_page(self, html: str):
        soup = BeautifulSoup(html, "lxml")

        self.create_readme(soup)

        tasks = soup.find_all("div", id=lambda x: x and x.startswith("task-num-"))

        self.create_task_files(tasks)


    def create_project(self, url: str):
        res = self.make_request(url)

        self.response = res

        self.scrap_the_page(res.text)

In [48]:
creator = ProjectCreator()

# creator.create_project("https://intranet.hbtn.io/projects/2275")

soup: BeautifulSoup
# soup = BeautifulSoup(creator.response.text, "lxml")

In [49]:
creator.create_task_files(soup)